In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import EarlyStopping, ModelCheckpoint


Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [9]:
weights_directory = "Weights/"

num_truck_brand = 9

dataset_directory = "Cropped_ClassifiedTruckType"

optimizer_name = 'adam'

In [10]:
batch_size = 4
image_height = 150
image_width = 150

In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(image_height, image_width, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(265, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

In [12]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_truck_brand))
model.add(Activation('softmax'))

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_8 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 74, 74, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
activation_9 (Activation)    (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 36, 36, 64)       

In [13]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer_name, metrics=['accuracy'])


model_name = weights_directory+optimizer_name+"_TruckBrand_Model_FullTruck"

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
mcp_save = ModelCheckpoint(model_name+'.h5', save_best_only=True, monitor='val_loss', verbose=1, mode='auto')

In [14]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        dataset_directory,  # this is the target directory
        target_size=(image_height, image_width),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        dataset_directory,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode='categorical')


Found 3059 images belonging to 9 classes.
Found 3059 images belonging to 9 classes.


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=800 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=20 // batch_size,
        callbacks=[early_stopping, mcp_save, TQDMNotebookCallback()])


Epoch 1/50


200/200 [==============================] - 613s 3s/step - loss: 2.1292 - acc: 0.4217 - val_loss: 1.1321 - val_acc: 0.7000

Epoch 00001: val_loss improved from inf to 1.13215, saving model to Weights/adam_TruckBrand_Model_FullTruck.h5
Epoch 2/50


200/200 [==============================] - 538s 3s/step - loss: 1.3210 - acc: 0.6088 - val_loss: 1.5105 - val_acc: 0.7000

Epoch 00002: val_loss did not improve from 1.13215
Epoch 3/50


200/200 [==============================] - 550s 3s/step - loss: 0.8896 - acc: 0.7337 - val_loss: 0.4216 - val_acc: 0.8000

Epoch 00003: val_loss improved from 1.13215 to 0.42163, saving model to Weights/adam_TruckBrand_Model_FullTruck.h5
Epoch 4/50


200/200 [==============================] - 454s 2s/step - loss: 0.8069 - acc: 0.7471 - val_loss: 1.2177 - val_acc: 0.8500

Epoch 00004: val_loss did not improve from 0.42163
Epoch 5/50


...

200/200 [==============================] - 452s 2s/step - loss: 0.5405 - acc: 0.8387 - val_loss: 0.9398 - val_acc: 0.7500

Epoch 00006: val_loss did not improve from 0.42163
Epoch 7/50


200/200 [==============================] - 474s 2s/step - loss: 0.4044 - acc: 0.8587 - val_loss: 0.1747 - val_acc: 0.9000

Epoch 00007: val_loss improved from 0.42163 to 0.17474, saving model to Weights/adam_TruckBrand_Model_FullTruck.h5
Epoch 8/50


200/200 [==============================] - 473s 2s/step - loss: 0.5171 - acc: 0.8663 - val_loss: 2.9838 - val_acc: 0.5500

Epoch 00008: val_loss did not improve from 0.17474
Epoch 9/50


200/200 [==============================] - 643s 3s/step - loss: 0.5509 - acc: 0.8525 - val_loss: 0.3290 - val_acc: 0.9000

Epoch 00009: val_loss did not improve from 0.17474
Epoch 10/50


 49/200 [======>.......................] - ETA: 9:43 - loss: 0.4513 - acc: 0.8673

In [ ]:
batch_size = 16

generator = test_datagen.flow_from_directory(
            dataset_directory,
            target_size=(150, 150),
            batch_size=batch_size,
            class_mode=None,  # this means our generator will only yield batches of data, no labels
            shuffle=False)  # our data will be in order, so all first 1000 images will be cats, then 1000 dogs
# the predict_generator method returns the output of a model, given
# a generator that yields batches of numpy data



In [ ]:
from keras.models import load_model

loaded_model = load_model(weights_directory+optimizer_name+"_TruckBrand_Model_FullTruck.h5")

In [ ]:
bottleneck_features_train = loaded_model.predict_generator(generator)


In [ ]:
import os

image = np.random.rand(150, 150, 3)
image = np.expand_dims(image, axis=0) 
predicted = loaded_model.predict(image)[0].argmax()

Truck_labels = [name for name in os.listdir("TruckTypeDataset/Train/")]
print(Truck_labels)
Truck_labels[predicted]

In [ ]:
score = loaded_model.evaluate_generator(validation_generator)

print("Loss: "+str(score[0]))
print("Accuracy: "+str(score[1]))